<h1 align = 'center'>Trabalho de Circuitos Elétricos II<h1/>

<p align = 'center'>
    <img src = '../docs/logo.jpeg'/>
<p/>

Este projeto tem como objetivo o estudo referente ao dimensionamento de um Banco de Capacitores.

Desenvolvedores:

- André Vitor Cadena de Oliveira
- Fábio Augusto Almeida Marçal
- Gabriel Nazário Gonçalves
- Lorenzo Carrera de Oliveira
- Mateus Pincho de Oliveira

---

In [10]:
# Importando os módulos utilizados...
import numpy as np
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv('curvas_de_carga_da_industria.csv') # Constrói um dataframe pandas com os dados fornecidos
df.index = np.full(24, '') # Esconde a indexação do dataframe

## Questão 1

Uma industria de médio porte identificou multas na fatura de energia elétrica 1 mês após 
instalar um gerador distribuído fotovoltaico para reduzir os custos. Um engenheiro eletricista, contratado 
pela industria, realizou a medição de potência na saída do gerador fotovoltaico, utilizando um analisador 
de energia Fluke 435. As curvas de potência ativa e reativa da indústria antes da instalação do gerador e 
a curva de potência injetada pelo gerador, discretizadas em períodos de 1 em 1 hora, estão apresentadas em `curvas_de_carga_da_industria.csv`. A partir dessas curvas, projete um **Quadro para Correção do Fator de Potência** , utilizando
um controlador de carga capaz de realizar a inserção de 6 células capacitivas em paralelo. Para o dimensionamento,
considere como fator de potência mínimo o valor mencionado no Modulo 7 do PRODIST/ANEEL. 

**OBS:** fator de potência mínimo exigido = $0.92$.

--- 

## Produção da Indústria

Primeiramente, avalia-se quanto de potência ativa a indústria de fato está consumindo. Como ela tem a sua disposição um gerador distribuído fotovoltaico, haverá um abatimento na potência total consumida, de modo que:

$$P_{total} = P_{consumida} - P_{gerador}$$

---

In [11]:
# Plotando as Curvas de Cargas Ativas

fig = go.Figure()

# Parâmetros do Gráfico
fig.update_layout(
    title='Curvas de Carga Ativas',
    xaxis_title='Hora do Dia',
    yaxis_title='Potência (kW)',
    legend_title='Cargas Ativas',
    plot_bgcolor='white',
    font=dict(
        family='Arial',
        size=15,
        color='black'
    ),
    xaxis=dict(
        gridcolor='lightgray',
        tickmode = 'linear',
        dtick = 1,
        range=[0,23]
    ),
    yaxis=dict(
        gridcolor='lightgray'
    )
)

# Calculando Potência Total
df['Potência Total'] = df['Potência Consumida'] - df['Potência Geradora']

for P in df.columns[1:4]:
    l = dict()
    if P == 'Potência Total':
        l = dict(
            width=4,
            dash='dash'
        )

    fig.add_trace(go.Scatter(
        x=df['Hora'],
        y=df[P],
        name=P,
        line=l
    )
)

fig.show()


## Cálculo do Fator de Potência 

O cálculo do fator de potência para cada hora do dia será dado por:

$${FP} = \cos(\arctan(\frac{Q_{reativo}}{P_{total}}))$$

Para cada horário, deve-se verificar se há a necessidade de corrigir o fator de potência.
   - Se ${FP} > {FP}_{min}$, sendo ${FP}_{min} = 0.94$, não há necessidade de corrigir o fator de potência;
   - Porém, se ${FP} < {FP}_{min}$, deve-se corrigir o fator de potência.

**OBS:** o fator de potência mínimo exigido é de $0.92$. Porém, para uma margem de erro mais segura, será utilizado $0.94$.

---

In [12]:
# Plotando as Curvas de Cargas Ativas

fig = go.Figure()

# Parâmetros do Gráfico
fig.update_layout(
    title='Evolução do Fator de Potência ao Longo do Dia',
    xaxis_title='Hora do Dia',
    yaxis_title='Fator de Potência',
    legend_title='Classificação',
    plot_bgcolor='white',
    font=dict(
        family='Arial',
        size=15,
        color='black'
    ),
    xaxis=dict(
        gridcolor='lightgray',
        tickmode = 'linear',
        dtick = 1,
        range=[0,23]
    ),
    yaxis=dict(
        gridcolor='lightgray',
        range=[0,1]
    )
)

# Calculando a evolução do Fator de Potência 
df['FP'] = (df['Reativo Indutivo']/df['Potência Total']).apply(np.arctan).apply(np.cos)

fig.add_hrect(
    y0=0.94, 
    y1=1.00, 
    line_width=0, 
    fillcolor='green', 
    opacity=0.2,
    layer='below')

fig.add_hrect(
    y0=0.92, 
    y1=0.94, 
    line_width=0, 
    fillcolor='yellow', 
    opacity=0.2,
    layer='below')

fig.add_hrect(
    y0=0.00, 
    y1=0.92, 
    line_width=0, 
    fillcolor='red', 
    opacity=0.2,
    layer='below')

fig.add_trace(go.Scatter(
    x=df['Hora'],
    y=df['FP'].apply(lambda x: x if x > 0.92 else None), # Somente os pontos que estão dentro da norma
    name='Nível Adequado',
    mode='markers',
    marker=dict(
            size=7,
            color='forestgreen'
        )
    )
)

fig.add_trace(go.Scatter(
    x=df['Hora'],
    y=df['FP'].apply(lambda x: x if x <= 0.92 else None), # Somente os pontos que estão fora do padrão
    name='Fora do Padrão',
    mode='markers',
    marker=dict(
            size=7,
            color='crimson'
        )
    )
)

fig.add_trace(go.Scatter(
    x=df['Hora'],
    y=np.full(24, 0.92),
    name='Fator Mínimo Exigido',
    line=dict(
            width=0.5,
            dash='dash',
            color='black'
        )
    )
)

fig.show()


## Controle do Fator de Potência 

Para o controle do Fator de Potência da indústria, será utilizado um banco de capacitores para corrigir o reativo produzido pela indústria e aumentar o fator de potência de modo que ${FP} \geq {FP}_{min} = 0.94$.

A quantidade de Potência Reativa Capacitiva necessária para aumentar o fator de potência é calculada por:


$$Q_{capacitivo} = P\cdot [\tan(\arccos({FP}))-\tan(\arccos({FP}_{min}))]$$

---

In [13]:
# Plotando as Curvas de Cargas Ativas

fig = go.Figure()

# Parâmetros do Gráfico
fig.update_layout(
    title='Correção do Fator de Potência via Banco de Capacitores',
    xaxis_title='Hora do Dia',
    yaxis_title='Potência Reativa Capacitiva (kVAr)',
    legend_title='Bancos de Capacitores',
    plot_bgcolor='white',
    font=dict(
        family='Arial',
        size=15,
        color='black'
    ),
    xaxis=dict(
        gridcolor='lightgray',
        tickmode = 'linear',
        dtick = 1,
        range=[0,23]
    ),
    yaxis=dict(
        gridcolor='lightgray'
    )
)

# Calculando a quantidade necessária de potência reativa capacitiva
df['Reativo Capacitivo Calculado'] = df['Potência Total']*(df['FP'].apply(np.arccos).apply(np.tan)-np.tan(np.arccos(0.94)))

# Desconsiderando valores negativos (ou seja, quando não é necessária a correção)
df['Reativo Capacitivo Calculado'] = df['Reativo Capacitivo Calculado'].apply(lambda x: None if x <= 0 else x)

fig.add_trace(go.Scatter(
    x=df['Hora'],
    y=df['Reativo Capacitivo Calculado'],
    name='Reativo Capacitivo Calculado',
    mode='markers',
    marker=dict(
        size=10
        )
    )
)

fig.show()
